<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.3-Stochastic_Gradient_Descent.ipynb)

# Stochastischer Gradientenabstieg (SGD)

In diesem Abschnitt werden wir den *stochastischen Gradientenabstieg* näher erläutern.
angelehnt an:
- [Dive into Deep Learning, Zhang, Aston and Lipton, Zachary C. and Li, Mu and Smola, Alexander J.](https://d2l.ai/index.html)
- [Dive into Deep Learning](https://arxiv.org/abs/2106.11342)

In [ ]:
# !pip install requests
# !pip install autograd
# !git clone https://github.com/dsgiitr/d2l-pytorch.git
# Homepage
# https://d2l.ai/

%matplotlib inline
import math
import matplotlib.pyplot as plt
import torch
import d2l
import numpy as np


## Stochastische Gradienten-Updates

Beim Deep Learning ist die Zielfunktion normalerweise der Durchschnitt der Verlustfunktionen für jedes Beispiel im Trainingsdatensatz. 

Gegeben ist ein Trainingsdatensatz von $n$ Beispielen. Nehmen wir an, dass $f_i(\mathbf{x})$ die Verlustfunktion 
in Bezug auf das Trainingsbeispiel mit dem Index $i$ ist, wobei $\mathbf{x}$ der Parametervektor ist.
Dann erhalten wir die Zielfunktion

$$f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n f_i(\mathbf{x}).$$

Der Gradient der Zielfunktion bei $\mathbf{x}$ wird berechnet als

$$\nabla f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n \nabla f_i(\mathbf{x}).$$

Bei Verwendung des Gradientenabstiegs beträgt der Rechenaufwand für jede Iteration der unabhängigen Variablen $\mathcal{O}(n)$, der linear mit $n$ wächst. Daher sind die Kosten des Gradientenabstiegs für jede Iteration höher, wenn der Trainingsdatensatz grösser ist.




Der stochastische Gradientenabstieg (SGD) reduziert die Rechenkosten bei jeder Iteration. Bei jeder Iteration des stochastischen Gradientenabstiegs wird **ein** Index $i\in\{1,\ldots, n\}$ für Datenbeispiele zufällig ausgewählt und der Gradient $\nabla f_i(\mathbf{x})$ zur Aktualisierung von $\mathbf{x}$ berechnet:

$$\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f_i(\mathbf{x}),$$

wobei $\eta$ die Lernrate ist. 
- Wir sehen, dass die Rechenkosten für jede Iteration von $\mathcal{O}(n)$ des Gradientenabstiegs auf die Konstante $\mathcal{O}(1)$ fallen. 
- Ausserdem möchten wir betonen, dass der stochastische Gradient $\nabla f_i(\mathbf{x})$ eine unverzerrte Schätzung des vollständigen Gradienten $\nabla f(\mathbf{x})$ ist, weil

$$\mathbb{E}_i \nabla f_i(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n \nabla f_i(\mathbf{x}) = \nabla f(\mathbf{x}).$$

Das bedeutet, dass der stochastische Gradient im Durchschnitt eine gute Schätzung (*erwartungstreuer Schätzer*) des Gradienten ist.

Nun werden wir ihn mit dem Gradientenabstieg vergleichen, indem wir dem Gradienten zufälliges Rauschen mit einem Mittelwert von 0 und einer Varianz von 1 hinzufügen, um einen stochastischen Gradientenabstieg zu simulieren.

In [ ]:
def f(x1, x2):
    return x1**2 + 2 * x2**2  # objective


def gradf(x1, x2):
    return (2 * x1, 4 * x2)  # gradient


In [ ]:
def sgd(x1, x2, s1, s2):  # simulate noisy gradient
    (g1, g2) = gradf(x1, x2)  # compute gradient
    (g1, g2) = (g1 + np.random.normal(0.1), g2 + np.random.normal(0.1))
    return (x1 - eta * lr() * g1, x2 - eta * lr() * g2, 0, 0)  # update variables


def train_2d(trainer):
    x1, x2, s1, s2 = -5, -2, 0, 0
    results = [(x1, x2)]
    for i in range(20):
        x1, x2, s1, s2 = trainer(x1, x2, s1, s2)
        results.append((x1, x2))
    print("epoch %d, x1 %f, x2 %f" % (i + 1, x1, x2))
    return results


In [ ]:
def constant_lr():
    return 1


eta = 0.1
lr = constant_lr
plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f, train_2d(sgd))


- Wie wir sehen können, ist die Trajektorie der Variablen im stochastischen Gradientenabstieg viel stärker verrauscht als diejenige, die wir beim Gradientenabstieg beobachtet haben. 
- Dies ist auf die stochastische Natur des Gradienten zurückzuführen. Das heisst, selbst wenn wir in der Nähe des Minimums ankommen, sind wir immer noch der Unsicherheit ausgesetzt, die durch den momentanen Gradienten über $\eta \nabla f_i(\mathbf{x})$ eingebracht wird. 
- Selbst nach 50 Schritten ist die Qualität immer noch nicht so gut. Noch schlimmer ist, dass sie sich auch nach zusätzlichen Schritten nicht verbessert (wir empfehlen Ihnen, mit einer grösseren Anzahl von Schritten zu experimentieren, um dies zu bestätigen). 
- Es bleibt uns also nur die Alternative, die **Lernrate $\eta$ zu ändern**. Wenn wir diese jedoch zu klein wählen, werden wir zunächst keine nennenswerten Fortschritte erzielen. Wenn wir sie andererseits zu gross wählen, werden wir, wie oben gesehen, keine gute Lösung erhalten. Die einzige Möglichkeit, diesen Zielkonflikt aufzulösen, besteht darin, die Lernrate im Laufe der Optimierung **dynamisch** zu verringern.

Dies ist auch der Grund für die Hinzufügung einer Lernratenfunktion `lr` in die Schrittfunktion `sgd`. Im obigen Beispiel sind alle Funktionen für die Planung der Lernrate inaktiv, da wir die zugehörige Funktion `lr` auf einen konstanten Wert setzen.



## Dynamische Lernrate

Das Ersetzen von $\eta$ durch eine zeitabhängige Lernrate $\eta(t)$ erhöht die Komplexität der Kontrolle der Konvergenz eines Optimierungsalgorithmus. Insbesondere müssen wir herausfinden, wie schnell $\eta$ abfallen soll. Wenn sie zu schnell ist, wird die Optimierung vorzeitig abgebrochen. Wenn er zu langsam abnimmt, verschwenden wir zu viel Zeit mit der Optimierung. Im Folgenden werden einige grundlegende Strategien vorgestellt, die bei der Anpassung von $\eta$ im Laufe der Zeit zum Einsatz kommen (wir werden später auf fortgeschrittenere Strategien eingehen):

$$
\begin{aligned}
    \eta(t) & = \eta_i \text{ if } t_i \leq t \leq t_{i+1} && \text{stückweise Konstante} \\
    \eta(t) & = \eta_0 \cdot e^{-\lambda t} && \text{exponentieller Zerfall} \\
    \eta(t) & = \eta_0 \cdot (\beta t + 1)^{-\alpha} && \text{polynomieller Zerfall}
\end{aligned}
$$

Im ersten *stückweise konstanten* Szenario verringern wir die Lernrate, z. B. immer dann, wenn der Fortschritt bei der Optimierung ins Stocken gerät. Dies ist eine übliche Strategie für das Training tiefer Netzwerke. Alternativ könnten wir die Lernrate durch einen *exponentiellen Zerfall* viel aggressiver verringern. Leider führt dies oft zu einem vorzeitigen Abbruch, bevor der Algorithmus konvergiert hat. Eine beliebte Wahl ist der *polynomiale Zerfall* mit $\alpha = 0.5$. Im Falle der konvexen Optimierung gibt es eine Reihe von Beweisen, die zeigen, dass sich diese Rate gut verhält. 

Schauen wir uns an, wie der exponentielle Zerfall in der Praxis aussieht.



In [ ]:
def exponential_lr():
    # Global variable that is defined outside this function and updated inside
    global t
    t += 1
    return math.exp(-0.1 * t)


In [ ]:
t = 1
lr = exponential_lr

plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f, train_2d(sgd))


Wie erwartet, wird die Varianz der Parameter erheblich reduziert. 
- Dies geschieht jedoch um den Preis, dass wir nicht zur optimalen Lösung $\mathbf{x} = (0, 0)$ konvergieren können.
- Selbst nach 1000 Iterationsschritten sind wir noch sehr weit von der optimalen Lösung entfernt. In der Tat konvergiert der Algorithmus überhaupt nicht. 
- Verwendet man dagegen einen polynomialen Zerfall, bei dem die Lernrate mit der inversen Quadratwurzel aus der Anzahl der Schritte abnimmt, wird die Konvergenz schon nach 50 Schritten besser.


In [ ]:
def polynomial_lr():
    # Global variable that is defined outside this function and updated inside
    global t
    t += 1
    return (1 + 0.1 * t) ** (-0.5)


In [ ]:
t = 1
lr = polynomial_lr
plt.figure(figsize=(10, 5))
d2l.show_trace_2d(f, train_2d(sgd))


Es gibt noch viele weitere Möglichkeiten, wie die **Lernrate $\eta$** festgelegt werden kann. 
- Wir könnten zum Beispiel mit einer kleinen Rate beginnen, dann schnell ansteigen und dann wieder abnehmen, wenn auch langsamer. 
- Wir könnten sogar zwischen kleineren und größeren Lernraten abwechseln. Es gibt eine grosse Vielfalt solcher Schemata. 

Im Folgenden wollen wir uns auf Lernratenpläne konzentrieren, für die eine umfassende theoretische Analyse möglich ist, d. h. auf Lernraten in einem konvexen Umfeld. Für allgemeine nichtkonvexe Probleme ist es sehr schwierig, sinnvolle Konvergenzgarantien zu erhalten, da die Minimierung nichtlinearer nichtkonvexer Probleme im Allgemeinen NP-schwer ist. Für einen Überblick siehe z.B. die ausgezeichneten [Vorlesungsunterlagen](https://www.stat.cmu.edu/~ryantibs/convexopt-F15/lectures/26-nonconvex.pdf) von Tibshirani 2015.










## Stochastische Gradienten und endliche Stichproben

Bislang haben wir uns mit dem Thema stochastischer Gradientenabstieg ein wenig schwer getan. Wir haben vorausgesetzt, dass wir Instanzen $x_i$, typischerweise mit Bezeichnungen $y_i$, aus einer Verteilung $p(x, y)$ ziehen und dass wir diese verwenden, um die Modellparameter auf irgendeine Weise zu aktualisieren. Insbesondere haben wir für einen endlichen Stichprobenumfang einfach argumentiert, dass die diskrete Verteilung 

$$p(x, y) = \frac{1}{n} \sum_{i=1}^n \delta_{x_i}(x) \delta_{y_i}(y)$$

für einige Funktionen $\delta_{x_i}$ und $\delta_{y_i}$  es uns erlaubt, einen stochastischen Gradientenabstieg darüber durchzuführen.

- Dies ist jedoch nicht das, was wir wirklich getan haben. In den simplen Beispielen in diesem Abschnitt haben wir einfach Rauschen zu einem ansonsten nicht stochastischen Gradienten hinzugefügt, d.h. wir haben so getan, als hätten wir Paare $(x_i, y_i)$. Es stellt sich heraus, dass dies hier gerechtfertigt ist (siehe die Übungen für eine detaillierte Diskussion). 



Noch beunruhigender ist, dass wir dies in allen früheren Diskussionen eindeutig nicht getan haben. Stattdessen haben wir über alle Instanzen *exakt einmal* iteriert. Um zu sehen, warum dies vorzuziehen ist, betrachten wir den umgekehrten Fall, nämlich dass wir $n$ Beobachtungen aus der diskreten Verteilung **mit Zurücklegen** abfragen. Die Wahrscheinlichkeit, ein Element $i$ zufällig auszuwählen, ist $1/n$. Es *mindestens* einmal zu wählen, ist also

$$P(\mathrm{choose~} i) = 1 - P(\mathrm{omit~} i) = 1 - (1-1/n)^n \approx 1-e^{-1} \approx 0.63.$$

Eine ähnliche Argumentation zeigt, dass die Wahrscheinlichkeit, eine Stichprobe (d.h. ein Trainingsbeispiel) *exakt einmal* auszuwählen, gegeben ist durch 

$${n \choose 1} \frac{1}{n} \left(1-\frac{1}{n}\right)^{n-1} = \frac{n}{n-1} \left(1-\frac{1}{n}\right)^{n} \approx e^{-1} \approx 0.37.$$ 

Dies führt zu einer erhöhten Varianz und einer geringeren Dateneffizienz im Vergleich zur Stichprobenziehung **ohne Zurücklegen**. Daher führen wir in der Praxis Letzteres durch (und dies ist die Standardwahl in diesem Kurs). Zuletzt sei noch angemerkt, dass wiederholte Durchläufe durch den Trainingsdatensatz diesen in einer *anderen* zufälligen Reihenfolge durchlaufen.



## Zusammenfassung

* Für konvexe Probleme lässt sich beweisen, dass der **stochastische Gradientenabstieg** bei einer breiten Auswahl von Lernraten zur **optimalen Lösung konvergiert**.
* Für Deep Learning ist dies im Allgemeinen nicht der Fall. Die Analyse konvexer Probleme gibt uns jedoch einen nützlichen Einblick in die Herangehensweise an die Optimierung, nämlich die **Lernrate schrittweise zu verringern**, wenn auch nicht zu schnell.
* Probleme treten auf, wenn die Lernrate zu klein oder zu gross ist. In der Praxis wird eine geeignete Lernrate oft erst nach mehreren Versuchen gefunden.
* Wenn der Trainingsdatensatz mehr Beispiele enthält, ist die Berechnung jeder Iteration für den **Gradientenabstieg teurer**, so dass in diesen Fällen der stochastische Gradientenabstieg vorzuziehen ist.
* **Optimalitätsgarantien** für den stochastischen Gradientenabstieg sind in nicht-konvexen Fällen im Allgemeinen nicht verfügbar, da die Anzahl der lokalen Minima, die überprüft werden müssen, exponentiell sein kann.



## Übungsaufgaben (optional)

1. Experimentieren Sie mit verschiedenen Lernratenschemata für den stochastischen Gradientenabstieg und mit verschiedenen Iterationszahlen. Stellen Sie insbesondere den Abstand von der optimalen Lösung $(0, 0)$ in Abhängigkeit von der Anzahl der Iterationen dar.
2. Beweisen Sie, dass für die Funktion $f(x_1, x_2) = x_1^2 + 2 x_2^2$ das Hinzufügen von normalem Rauschen zum Gradienten äquivalent zur Minimierung einer Verlustfunktion $f(\mathbf{x}, \mathbf{w}) = (x_1 - w_1)^2 + 2 (x_2 - w_2)^2$ ist, wobei $\mathbf{x}$ aus einer Normalverteilung gezogen wird.
3. Vergleichen Sie die Konvergenz des stochastischen Gradientenabstiegs bei Stichproben aus $\{(x_1, y_1), \ldots, (x_n, y_n)\}$ mit Ersetzung und bei Stichproben ohne Ersetzung.
4. Wie würden Sie den Löser des stochastischen Gradientenabstiegs ändern, wenn ein Gradient (oder vielmehr eine damit verbundene Koordinate) durchweg größer als alle anderen Gradienten wäre?
5. Nehmen Sie an, dass $f(x) = x^2 (1 + \sin x)$. Wie viele lokale Minima hat $f$? Kann man $f$ so verändern, dass man alle lokalen Minima auswerten muss, um es zu minimieren?